### Instalar librerías, ya creado el entorno y verificado que se puede ejecutar el script de Python

In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk


### Importar archivo remuneraciones extención XLSX, usando como encabezado la primera fila útil

In [87]:
archivos = r"D:\Stella\Documentos\POLITECNICO\2 año\Aprendizaje automático\Insumos Parcial\Insumos para el modelo\21_1_01_Remuneracion-promedio-de-los-trabajadores-registrados-del-sector-privado-segun-rama-de-actividad.xlsx"

df_remuneraciones = pd.read_excel(archivos, engine="openpyxl", header=2,)
print(df_remuneraciones.head())

  Rama de Actividad                                       Unnamed: 1  \
0                 A      AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA   
1                 1                         Agricultura y ganadería    
2                 2              Silvicultura, extracción de madera    
3                 B                        PESCA Y SERVICIOS CONEXOS   
4                 5   Pesca y actividades relacionadas con la pesca    

  1995-01-01 00:00:00 1995-02-01 00:00:00 1995-03-01 00:00:00  \
0                 797                 738                 853   
1                 923                 821                 974   
2                 462                 517                 552   
3                1282                1229                1360   
4                1282                1229                1360   

  1995-04-01 00:00:00 1995-05-01 00:00:00 1995-06-01 00:00:00  \
0                 527                 469                1049   
1                 528                 461     

### Seleccionar las columnas útiles

In [88]:
print(df_remuneraciones.columns.tolist())
print(type(df_remuneraciones.columns))

['Rama de Actividad', 'Unnamed: 1', datetime.datetime(1995, 1, 1, 0, 0), datetime.datetime(1995, 2, 1, 0, 0), datetime.datetime(1995, 3, 1, 0, 0), datetime.datetime(1995, 4, 1, 0, 0), datetime.datetime(1995, 5, 1, 0, 0), datetime.datetime(1995, 6, 1, 0, 0), datetime.datetime(1995, 7, 1, 0, 0), datetime.datetime(1995, 8, 1, 0, 0), datetime.datetime(1995, 9, 1, 0, 0), datetime.datetime(1995, 10, 1, 0, 0), datetime.datetime(1995, 11, 1, 0, 0), datetime.datetime(1995, 12, 1, 0, 0), datetime.datetime(1996, 1, 1, 0, 0), datetime.datetime(1996, 2, 1, 0, 0), datetime.datetime(1996, 3, 1, 0, 0), datetime.datetime(1996, 4, 1, 0, 0), datetime.datetime(1996, 5, 1, 0, 0), datetime.datetime(1996, 6, 1, 0, 0), datetime.datetime(1996, 7, 1, 0, 0), datetime.datetime(1996, 8, 1, 0, 0), datetime.datetime(1996, 9, 1, 0, 0), datetime.datetime(1996, 10, 1, 0, 0), datetime.datetime(1996, 11, 1, 0, 0), datetime.datetime(1996, 12, 1, 0, 0), datetime.datetime(1997, 1, 1, 0, 0), datetime.datetime(1997, 2, 1, 0, 

In [89]:
# Cambiar el nombre de las columnas para que sean más descriptivas o fáciles de usar
# Por ejemplo, renombrar las dos primeras columnas y dejar las fechas igual

df_remuneraciones = df_remuneraciones.rename(columns={
    df_remuneraciones.columns[0]: "Codigo",
    df_remuneraciones.columns[1]: "Rama_Actividad"
})

print(df_remuneraciones.columns[:10])  # Mostrar los primeros 10 nombres de columnas para verificar

Index([           'Codigo',    'Rama_Actividad', 1995-01-01 00:00:00,
       1995-02-01 00:00:00, 1995-03-01 00:00:00, 1995-04-01 00:00:00,
       1995-05-01 00:00:00, 1995-06-01 00:00:00, 1995-07-01 00:00:00,
       1995-08-01 00:00:00],
      dtype='object')


### Renombrar las columnas, para poder luego seleccionar aquellas que formen parte del análisis y posterior entrenamiento del modelo

In [90]:
# Renombrar las columnas desde "Rama_Actividad" en adelante para que tengan formato de fecha YYYY-MM
# Primero identificamos el índice de la columna "Rama_Actividad"
idx = df_remuneraciones.columns.get_loc("Rama_Actividad")

# Las columnas de fechas empiezan en el siguiente índice
fecha_cols = df_remuneraciones.columns[(idx+1):]

# Creamos nuevos nombres en formato YYYY-MM
nuevos_nombres = {}
for col in fecha_cols:
    try:
        # Intentar convertir el nombre de la columna a formato fecha
        fecha = pd.to_datetime(col, format='%Y-%m')
        nuevos_nombres[col] = fecha.strftime('%Y-%m')
    except Exception:
        # Si no se puede convertir, dejar el nombre original
        nuevos_nombres[col] = col

# Renombrar las columnas en el DataFrame
df_remuneraciones = df_remuneraciones.rename(columns=nuevos_nombres)

print(df_remuneraciones.columns[idx:idx+10])  # Mostrar los primeros 10 nombres después de "Rama_Actividad"

Index(['Rama_Actividad', '1995-01', '1995-02', '1995-03', '1995-04', '1995-05',
       '1995-06', '1995-07', '1995-08', '1995-09'],
      dtype='object')


In [91]:
# Seleccionar las columnas "Codigo", "Rama_Actividad" y desde "2017-12" hasta el final
cols = df_remuneraciones.columns.tolist()
start_idx = cols.index("2017-12")
selected_cols = ["Codigo", "Rama_Actividad"] + cols[start_idx:]
df_remuneraciones_sel = df_remuneraciones[selected_cols]
print(df_remuneraciones_sel.head())

  Codigo                                   Rama_Actividad 2017-12 2018-01  \
0      A      AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA   29477   20495   
1      1                         Agricultura y ganadería    30861   21524   
2      2              Silvicultura, extracción de madera    22224   14432   
3      B                        PESCA Y SERVICIOS CONEXOS   72125   56527   
4      5   Pesca y actividades relacionadas con la pesca    72125   56527   

  2018-02 2018-03 2018-04 2018-05 2018-06 2018-07  ...  2024-03  2024-04  \
0   20629   22372   22193   23158   33354   23985  ...   678022   794914   
1   22011   24093   24166   25689   36915   26425  ...   760692   905057   
2   13803   15061   13702   13125   20094   13129  ...   444725   454192   
3   53876   51386   56918   62512   99129   57314  ...  2503400  2329619   
4   53876   51386   56918   62512   99129   57314  ...  2503400  2329619   

   2024-05  2024-06  2024-07  2024-08  2024-09  2024-10  2024-11  2024-12  
0   

### Hasta acá parece que el primer DB está ok, ahora podría ver algún gráfico de correlación para explorar datos
*podría ver si hay registros nulos o duplicados*

In [92]:
# Verificar registros nulos
print("Cantidad de valores nulos por columna:")
print(df_remuneraciones_sel.isnull().sum())

# Verificar registros duplicados
duplicados = df_remuneraciones_sel.duplicated()
print(f"\nCantidad de filas duplicadas: {duplicados.sum()}")

Cantidad de valores nulos por columna:
Codigo            0
Rama_Actividad    3
2017-12           1
2018-01           1
2018-02           1
                 ..
2024-08           1
2024-09           1
2024-10           1
2024-11           1
2024-12           1
Length: 87, dtype: int64

Cantidad de filas duplicadas: 0


In [93]:
print(df_remuneraciones['Rama_Actividad'].dtype)
print(df_remuneraciones['Rama_Actividad'].unique())

object
['AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA' ' Agricultura y ganadería '
 ' Silvicultura, extracción de madera ' 'PESCA Y SERVICIOS CONEXOS'
 ' Pesca y actividades relacionadas con la pesca '
 'EXPLOTACION  DE  MINAS  Y  CANTERAS'
 ' Extracción de petróleo crudo y gas natural '
 ' Extracción de minerales metalíferos '
 ' Explotación de otras minas y canteras ' 'INDUSTRIA MANUFACTURERA'
 ' Alimentos ' ' Tabaco ' ' Productos textiles ' ' Confecciones '
 ' Cuero y calzado ' ' Madera ' ' Papel ' ' Edición '
 ' Productos de petróleo ' ' Productos químicos '
 ' Productos de caucho y plástico ' ' Otros minerales no metálicos '
 ' Metales comunes ' ' Otros productos de metal ' ' Maquinaria y equipo '
 ' Maquinaria de oficina ' ' Aparatos eléctricos ' ' Radio y televisión '
 ' Instrumentos médicos ' ' Automotores ' ' Otros equipo de transporte '
 ' Muebles ' ' Reciclamiento de desperdicios y desechos '
 'ELECTRICIDAD, GAS Y AGUA' ' Electricidad, gas y agua '
 ' Captación, depuración y 

In [94]:
# Para corregir errores en el DataFrame, primero identificamos filas no válidas (como las de totales, fuentes o confidencialidad)
# Eliminamos filas donde 'Codigo' o 'Rama_Actividad' no sean válidas

# Filtrar filas donde 'Codigo' y 'Rama_Actividad' no sean nulos y 'Codigo' no sea 'Total' ni contenga 'Fuente' o 'Dato confidencial'
df_remuneraciones_sel_clean = df_remuneraciones_sel[
    df_remuneraciones_sel['Codigo'].notnull() &
    df_remuneraciones_sel['Rama_Actividad'].notnull() &
    (~df_remuneraciones_sel['Codigo'].astype(str).str.contains('Total|Fuente|Dato confidencial', case=False, na=False))
].copy()

# Resetear el índice
df_remuneraciones_sel_clean.reset_index(drop=True, inplace=True)

print(df_remuneraciones_sel_clean.head())

  Codigo                                   Rama_Actividad 2017-12 2018-01  \
0      A      AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA   29477   20495   
1      1                         Agricultura y ganadería    30861   21524   
2      2              Silvicultura, extracción de madera    22224   14432   
3      B                        PESCA Y SERVICIOS CONEXOS   72125   56527   
4      5   Pesca y actividades relacionadas con la pesca    72125   56527   

  2018-02 2018-03 2018-04 2018-05 2018-06 2018-07  ...  2024-03  2024-04  \
0   20629   22372   22193   23158   33354   23985  ...   678022   794914   
1   22011   24093   24166   25689   36915   26425  ...   760692   905057   
2   13803   15061   13702   13125   20094   13129  ...   444725   454192   
3   53876   51386   56918   62512   99129   57314  ...  2503400  2329619   
4   53876   51386   56918   62512   99129   57314  ...  2503400  2329619   

   2024-05  2024-06  2024-07  2024-08  2024-09  2024-10  2024-11  2024-12  
0   

In [95]:
from sklearn.preprocessing import OneHotEncoder

# Crear el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, dtype=int)

# Ajustar y transformar la columna 'Rama_Actividad'
encoded = encoder.fit_transform(df_remuneraciones_sel_clean[['Rama_Actividad']])

# Crear un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Rama_Actividad']))

# Concatenar el DataFrame original (sin 'Rama_Actividad') con el codificado
df_remuneraciones_encoded = pd.concat(
    [df_remuneraciones_sel_clean.drop('Rama_Actividad', axis=1).reset_index(drop=True), encoded_df], axis=1
)

print(df_remuneraciones_encoded.head())

  Codigo 2017-12 2018-01 2018-02 2018-03 2018-04 2018-05 2018-06 2018-07  \
0      A   29477   20495   20629   22372   22193   23158   33354   23985   
1      1   30861   21524   22011   24093   24166   25689   36915   26425   
2      2   22224   14432   13803   15061   13702   13125   20094   13129   
3      B   72125   56527   53876   51386   56918   62512   99129   57314   
4      5   72125   56527   53876   51386   56918   62512   99129   57314   

  2018-08  ... Rama_Actividad_EXPLOTACION  DE  MINAS  Y  CANTERAS  \
0   22212  ...                                                  0   
1   23296  ...                                                  0   
2   17231  ...                                                  0   
3   75601  ...                                                  0   
4   75601  ...                                                  0   

  Rama_Actividad_HOTELERIA Y RESTAURANTES   \
0                                        0   
1                                   

Crear el Dataframe para la base de datos de la variacion de precios

In [96]:
df_precios = pd.read_excel("17_1_04_IPC_variaciones_interanuales_segun_divisiones_de_la_canasta_categorias_bienes_y_servicios._Patagonia.xlsx", engine="openpyxl", header=3,skiprows=2)



In [97]:
df_precios.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89
0,Nivel general,23.5,23.5,23.9,25.1,26.0,26.3,29.1,31.2,33.7,...,264.2,239.7,209.1,195.0,172.2,124.7,85.5,72.0,60.9,52.3
1,Alimentos y bebidas no alcohólicas,16.9,17.4,18.2,18.7,19.8,20.5,25.5,29.6,33.1,...,283.0,252.9,209.6,193.1,165.0,111.0,69.1,61.0,50.4,46.9
2,Bebidas alcohólicas y tabaco,22.5,23.7,20.7,20.3,18.8,19.0,18.5,18.8,19.2,...,255.9,239.6,208.9,189.4,173.0,133.0,93.8,65.0,49.4,44.8
3,Prendas de vestir y calzado,14.6,13.2,12.6,13.9,17.6,17.3,17.5,17.2,17.1,...,193.9,176.0,151.3,139.8,122.1,95.4,76.4,61.8,48.8,43.7
4,"Vivienda, agua, electricidad y otros combustibles",48.8,48.0,46.7,49.3,52.2,47.4,46.3,45.7,45.5,...,306.8,309.3,286.4,290.1,290.4,268.1,223.7,202.5,187.0,112.0


In [98]:
##ver nombre de las columnas
print(df_precios.columns.tolist())

['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unn

In [99]:
# Renombrar la primera columna
df_precios = df_precios.rename(columns={'Unnamed: 0': 'Categoria'})

# Obtener los nombres de columnas actuales
cols = df_precios.columns.tolist()

# Encontrar el índice de 'Unnamed: 1'
idx_fecha = cols.index('Unnamed: 1')



In [100]:
# Generar nuevos nombres para las columnas de fechas
nuevos_nombres = {}
for i, col in enumerate(cols[idx_fecha:], start=0):
    # Asignar nombres de fecha a partir de '2017-12'
    fecha = pd.to_datetime('2017-12') + pd.DateOffset(months=i)
    nuevos_nombres[col] = fecha.strftime('%Y-%m')

# Renombrar las columnas en el DataFrame
df_precios = df_precios.rename(columns=nuevos_nombres)

print(df_precios.columns.tolist())

['Categoria', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']


In [101]:
df_precios.head()

,Categoria,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04
0,Nivel general,23.5,23.5,23.9,25.1,26.0,26.3,29.1,31.2,33.7,...,264.2,239.7,209.1,195.0,172.2,124.7,85.5,72.0,60.9,52.3
1,Alimentos y bebidas no alcohólicas,16.9,17.4,18.2,18.7,19.8,20.5,25.5,29.6,33.1,...,283.0,252.9,209.6,193.1,165.0,111.0,69.1,61.0,50.4,46.9
2,Bebidas alcohólicas y tabaco,22.5,23.7,20.7,20.3,18.8,19.0,18.5,18.8,19.2,...,255.9,239.6,208.9,189.4,173.0,133.0,93.8,65.0,49.4,44.8
3,Prendas de vestir y calzado,14.6,13.2,12.6,13.9,17.6,17.3,17.5,17.2,17.1,...,193.9,176.0,151.3,139.8,122.1,95.4,76.4,61.8,48.8,43.7
4,"Vivienda, agua, electricidad y otros combustibles",48.8,48.0,46.7,49.3,52.2,47.4,46.3,45.7,45.5,...,306.8,309.3,286.4,290.1,290.4,268.1,223.7,202.5,187.0,112.0


In [102]:
# Detectar valores nulos en df_precios
print("Cantidad de valores nulos por columna en df_precios:")
print(df_precios.isnull().sum())

# Detectar filas duplicadas en df_precios
duplicados_precios = df_precios.duplicated()
print(f"\nCantidad de filas duplicadas en df_precios: {duplicados_precios.sum()}")

Cantidad de valores nulos por columna en df_precios:
Categoria    4
2017-12      8
2018-01      8
2018-02      8
2018-03      8
            ..
2024-12      8
2025-01      8
2025-02      8
2025-03      8
2025-04      8
Length: 90, dtype: int64

Cantidad de filas duplicadas en df_precios: 3


In [103]:
print("Cantidad de filas en df_precios:", len(df_precios))

Cantidad de filas en df_precios: 26


In [104]:
df_precios.tail()

,Categoria,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04
21,Servicios,35.6,34.8,35.0,36.5,36.7,35.8,35.5,34.7,36.7,...,270.7,268.9,251.2,248.3,237.9,216.7,178.6,159.1,136.0,101.1
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,"Nota: A partir del mes de junio de 2017, el IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Fuente: Instituto Nacional de Estadística y Ce...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
print(df_precios['Categoria'].unique())


['Nivel general' 'Alimentos y bebidas no alcohólicas'
 'Bebidas alcohólicas y tabaco' 'Prendas de vestir y calzado'
 'Vivienda, agua, electricidad y otros combustibles'
 'Equipamiento y mantenimiento del hogar' 'Salud' 'Transporte'
 'Comunicación' 'Recreación y cultura' 'Educación'
 'Restaurantes y hoteles' 'Bienes y servicios varios' nan 'Categorías '
 'Estacional' 'Núcleo' 'Regulados' 'Bienes y servicios' 'Bienes'
 'Servicios'
 'Nota: A partir del mes de junio de 2017, el INDEC adoptó el Clasificador del Consumo Individual por Finalidad (COICOP, por sus siglas en inglés: Classification of Individual Consumption According to\xa0 Purpose) de Naciones Unidas, 1999.\nLos índices están redondeados a 4 decimales. Esto puede producir pequeñas discrepancias con las variaciones publicadas, que se corresponden con las del informe técnico.'
 'Fuente: Instituto Nacional de Estadística y Censos (INDEC)']


In [106]:
df_precios.head(24)


,Categoria,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04
0,Nivel general,23.5,23.5,23.9,25.1,26.0,26.3,29.1,31.2,33.7,...,264.2,239.7,209.1,195.0,172.2,124.7,85.5,72.0,60.9,52.3
1,Alimentos y bebidas no alcohólicas,16.9,17.4,18.2,18.7,19.8,20.5,25.5,29.6,33.1,...,283.0,252.9,209.6,193.1,165.0,111.0,69.1,61.0,50.4,46.9
2,Bebidas alcohólicas y tabaco,22.5,23.7,20.7,20.3,18.8,19.0,18.5,18.8,19.2,...,255.9,239.6,208.9,189.4,173.0,133.0,93.8,65.0,49.4,44.8
3,Prendas de vestir y calzado,14.6,13.2,12.6,13.9,17.6,17.3,17.5,17.2,17.1,...,193.9,176.0,151.3,139.8,122.1,95.4,76.4,61.8,48.8,43.7
4,"Vivienda, agua, electricidad y otros combustibles",48.8,48.0,46.7,49.3,52.2,47.4,46.3,45.7,45.5,...,306.8,309.3,286.4,290.1,290.4,268.1,223.7,202.5,187.0,112.0
5,Equipamiento y mantenimiento del hogar,15.9,16.1,17.7,21.7,23.2,23.4,26.6,30.1,34.8,...,257.7,226.3,193.4,173.6,141.9,92.0,54.9,42.4,40.5,31.7
6,Salud,28.1,24.3,23.8,24.6,25.2,26.9,29.7,29.4,30.9,...,278.9,241.3,220.9,221.4,190.2,122.8,88.3,71.2,58.7,49.8
7,Transporte,27.1,26.8,29.0,28.5,30.3,33.4,39.9,44.4,49.5,...,261.7,233.4,216.9,194.4,173.0,113.7,67.5,52.3,38.6,35.9
8,Comunicación,28.7,27.2,32.3,33.1,24.6,28.0,28.3,28.6,38.1,...,297.0,299.6,283.9,254.4,220.2,193.4,142.2,100.8,82.9,65.9
9,Recreación y cultura,24.5,27.9,27.9,27.5,24.3,23.8,26.1,29.0,29.5,...,240.7,201.9,168.7,156.1,135.8,96.5,69.9,59.4,52.8,43.6


In [107]:
# Filtrar las filas útiles: desde 'Nivel general' hasta 'Servicios'
start_idx = df_precios[df_precios['Categoria'] == 'Nivel general'].index[0]
end_idx = df_precios[df_precios['Categoria'] == 'Servicios'].index[0]
df_precios_limpio = df_precios.loc[start_idx:end_idx].reset_index(drop=True)

# Ver los primeros datos para asegurarte
print(df_precios_limpio.head())

# Contar los nulos por columna
nulos_por_columna = df_precios_limpio.isna().sum()

# Contar nulos totales
total_nulos = df_precios_limpio.isna().sum().sum()

print("Nulos por columna:\n", nulos_por_columna)
print("\nTotal de nulos:", total_nulos)

df_precios_limpio = df_precios_limpio.dropna(axis=1, how='all')
df_precios_limpio.head(22)


                                           Categoria  2017-12  2018-01  \
0                                      Nivel general     23.5     23.5   
1                 Alimentos y bebidas no alcohólicas     16.9     17.4   
2                       Bebidas alcohólicas y tabaco     22.5     23.7   
3                        Prendas de vestir y calzado     14.6     13.2   
4  Vivienda, agua, electricidad y otros combustibles     48.8     48.0   

   2018-02  2018-03  2018-04  2018-05  2018-06  2018-07  2018-08  ...  \
0     23.9     25.1     26.0     26.3     29.1     31.2     33.7  ...   
1     18.2     18.7     19.8     20.5     25.5     29.6     33.1  ...   
2     20.7     20.3     18.8     19.0     18.5     18.8     19.2  ...   
3     12.6     13.9     17.6     17.3     17.5     17.2     17.1  ...   
4     46.7     49.3     52.2     47.4     46.3     45.7     45.5  ...   

   2024-07  2024-08  2024-09  2024-10  2024-11  2024-12  2025-01  2025-02  \
0    264.2    239.7    209.1    195.0  

,Categoria,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04
0,Nivel general,23.5,23.5,23.9,25.1,26.0,26.3,29.1,31.2,33.7,...,264.2,239.7,209.1,195.0,172.2,124.7,85.5,72.0,60.9,52.3
1,Alimentos y bebidas no alcohólicas,16.9,17.4,18.2,18.7,19.8,20.5,25.5,29.6,33.1,...,283.0,252.9,209.6,193.1,165.0,111.0,69.1,61.0,50.4,46.9
2,Bebidas alcohólicas y tabaco,22.5,23.7,20.7,20.3,18.8,19.0,18.5,18.8,19.2,...,255.9,239.6,208.9,189.4,173.0,133.0,93.8,65.0,49.4,44.8
3,Prendas de vestir y calzado,14.6,13.2,12.6,13.9,17.6,17.3,17.5,17.2,17.1,...,193.9,176.0,151.3,139.8,122.1,95.4,76.4,61.8,48.8,43.7
4,"Vivienda, agua, electricidad y otros combustibles",48.8,48.0,46.7,49.3,52.2,47.4,46.3,45.7,45.5,...,306.8,309.3,286.4,290.1,290.4,268.1,223.7,202.5,187.0,112.0
5,Equipamiento y mantenimiento del hogar,15.9,16.1,17.7,21.7,23.2,23.4,26.6,30.1,34.8,...,257.7,226.3,193.4,173.6,141.9,92.0,54.9,42.4,40.5,31.7
6,Salud,28.1,24.3,23.8,24.6,25.2,26.9,29.7,29.4,30.9,...,278.9,241.3,220.9,221.4,190.2,122.8,88.3,71.2,58.7,49.8
7,Transporte,27.1,26.8,29.0,28.5,30.3,33.4,39.9,44.4,49.5,...,261.7,233.4,216.9,194.4,173.0,113.7,67.5,52.3,38.6,35.9
8,Comunicación,28.7,27.2,32.3,33.1,24.6,28.0,28.3,28.6,38.1,...,297.0,299.6,283.9,254.4,220.2,193.4,142.2,100.8,82.9,65.9
9,Recreación y cultura,24.5,27.9,27.9,27.5,24.3,23.8,26.1,29.0,29.5,...,240.7,201.9,168.7,156.1,135.8,96.5,69.9,59.4,52.8,43.6


In [108]:
df_precios_limpio = df_precios_limpio.dropna(axis=0, how='any').reset_index(drop=True)
print(df_precios_limpio)

                                            Categoria  2017-12  2018-01  \
0                                       Nivel general     23.5     23.5   
1                  Alimentos y bebidas no alcohólicas     16.9     17.4   
2                        Bebidas alcohólicas y tabaco     22.5     23.7   
3                         Prendas de vestir y calzado     14.6     13.2   
4   Vivienda, agua, electricidad y otros combustibles     48.8     48.0   
5              Equipamiento y mantenimiento del hogar     15.9     16.1   
6                                               Salud     28.1     24.3   
7                                          Transporte     27.1     26.8   
8                                        Comunicación     28.7     27.2   
9                                Recreación y cultura     24.5     27.9   
10                                          Educación     29.2     21.1   
11                             Restaurantes y hoteles     21.7     22.6   
12                       

### El Dataframe sin nulos de IPC se llama df_precios_limpio


In [109]:
df_tarjetas= pd.read_excel("Series_anual_2023_20240507.xlsx",sheet_name='Pagos minoristas')

c:\Users\usuario\venv\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Crec absoluto MP 1er gráfico'!$A:$X.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\usuario\venv\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Participación MP 2do graf'!$A:$W.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [110]:
print(df_tarjetas.columns.tolist())

['Unnamed: 0', 'Cheques físicos compensados', 'Unnamed: 2', 'Unnamed: 3', 'Cheques electrónicos compensados', 'Unnamed: 5', 'Unnamed: 6', 'Transferencias inmediatas "push" administradas por las redes de transferencias electrónicas de fondos', 'Unnamed: 8', 'Unnamed: 9', 'Transferencias inmediatas "push" intra PSPCP', 'Unnamed: 11', 'Unnamed: 12', 'Transferencias inmediatas "pull" administradas por las redes de transferencias electrónicas de fondos', 'Unnamed: 14', 'Unnamed: 15', 'Transferencias de alto valor entre empresas', 'Unnamed: 17', 'Unnamed: 18', 'Transferencias en lote', 'Unnamed: 20', 'Unnamed: 21', 'Pagos con transferencias', 'Unnamed: 23', 'Unnamed: 24', 'Pagos con transferencias intra PSPCP', 'Unnamed: 26', 'Unnamed: 27', 'Débitos directos', 'Unnamed: 29', 'Unnamed: 30', 'Débito inmediato', 'Unnamed: 32', 'Unnamed: 33', 'Tarjetas de crédito', 'Unnamed: 35', 'Unnamed: 36', 'Tarjetas de débito', 'Unnamed: 38', 'Unnamed: 39', 'Tarjetas prepagas', 'Unnamed: 41', 'Unnamed: 42']

In [111]:
df_tarjetas_credito = df_tarjetas[['Unnamed: 0', 'Tarjetas de crédito', 'Unnamed: 35', 'Unnamed: 36']]
print(df_tarjetas_credito.head())

            Unnamed: 0 Tarjetas de crédito         Unnamed: 35  \
0                Fecha            Cantidad       Monto nominal   
1  2017-01-31 00:00:00            70383415  72229953417.461838   
2  2017-02-28 00:00:00            56753124  55963552050.587135   
3  2017-03-31 00:00:00            76609982  76279350650.953857   
4  2017-04-30 00:00:00            66385229  67998093275.004524   

                Unnamed: 36  
0  Monto en pesos dic. 2016  
1        71102341385.430298  
2        53974119962.875511  
3        71861587686.347931  
4        62402636103.511002  


In [112]:
df_tarjetas_credito.head(25)

,Unnamed: 0,Tarjetas de crédito,Unnamed: 35,Unnamed: 36
0,Fecha,Cantidad,Monto nominal,Monto en pesos dic. 2016
1,2017-01-31 00:00:00,70383415,72229953417.461838,71102341385.430298
2,2017-02-28 00:00:00,56753124,55963552050.587135,53974119962.875511
3,2017-03-31 00:00:00,76609982,76279350650.953857,71861587686.347931
4,2017-04-30 00:00:00,66385229,67998093275.004524,62402636103.511002
5,2017-05-31 00:00:00,72081744,77842127934.365128,70426180682.33461
6,2017-06-30 00:00:00,71931743,78028200603.128601,69762901340.956146
7,2017-07-31 00:00:00,68745930,80517085457.91217,70762353502.838821
8,2017-08-31 00:00:00,68222058,80107205511.866501,69427878646.361786
9,2017-09-30 00:00:00,68217790,78900533509.19928,67108325636.652359


In [113]:
# Eliminar las primeras 12 filas (por ejemplo, si son encabezados o datos no útiles)
df_tarjetas_credito = df_tarjetas_credito.drop(index=range(0, 12)).reset_index(drop=True)
print(df_tarjetas_credito.head())

            Unnamed: 0 Tarjetas de crédito         Unnamed: 35  \
0  2017-12-31 00:00:00            78243669  98697152418.107941   
1  2018-01-31 00:00:00            73758981  92136264258.031982   
2  2018-02-28 00:00:00            70494669  86155014922.501984   
3  2018-03-31 00:00:00            72407230  92311999328.511993   
4  2018-04-30 00:00:00            74657605  96804279009.734222   

          Unnamed: 36  
0   79087045070.58577  
1  72554695227.238327  
2  66242209341.262451  
3  69352557693.761475  
4   70788613927.87355  


In [114]:
df_tarjetas_credito.head(20)

,Unnamed: 0,Tarjetas de crédito,Unnamed: 35,Unnamed: 36
0,2017-12-31 00:00:00,78243669,98697152418.107941,79087045070.58577
1,2018-01-31 00:00:00,73758981,92136264258.031982,72554695227.238327
2,2018-02-28 00:00:00,70494669,86155014922.501984,66242209341.262451
3,2018-03-31 00:00:00,72407230,92311999328.511993,69352557693.761475
4,2018-04-30 00:00:00,74657605,96804279009.734222,70788613927.87355
5,2018-05-31 00:00:00,78065720,107878153664.534225,77282537891.180939
6,2018-06-30 00:00:00,74421150,103059418969.994217,71171026868.487701
7,2018-07-31 00:00:00,76032914,107613161087.435852,72080115077.929337
8,2018-08-31 00:00:00,85829190,119190742247.155838,76845989351.075378
9,2018-09-30 00:00:00,79790832,110978438313.96582,67162660420.717117


In [115]:
print(df_tarjetas_credito.iloc[0])

Unnamed: 0             2017-12-31 00:00:00
Tarjetas de crédito               78243669
Unnamed: 35             98697152418.107941
Unnamed: 36              79087045070.58577
Name: 0, dtype: object


In [116]:
df_tarjetas_credito.columns = ['Fecha', 'Cantidad', 'Monto nominal', 'Monto en pesos dic.2016']


In [117]:
df_tarjetas_credito.head(12)

,Fecha,Cantidad,Monto nominal,Monto en pesos dic.2016
0,2017-12-31 00:00:00,78243669,98697152418.107941,79087045070.58577
1,2018-01-31 00:00:00,73758981,92136264258.031982,72554695227.238327
2,2018-02-28 00:00:00,70494669,86155014922.501984,66242209341.262451
3,2018-03-31 00:00:00,72407230,92311999328.511993,69352557693.761475
4,2018-04-30 00:00:00,74657605,96804279009.734222,70788613927.87355
5,2018-05-31 00:00:00,78065720,107878153664.534225,77282537891.180939
6,2018-06-30 00:00:00,74421150,103059418969.994217,71171026868.487701
7,2018-07-31 00:00:00,76032914,107613161087.435852,72080115077.929337
8,2018-08-31 00:00:00,85829190,119190742247.155838,76845989351.075378
9,2018-09-30 00:00:00,79790832,110978438313.96582,67162660420.717117


In [118]:
df_remu_long = pd.melt(df_remuneraciones_encoded,
                       id_vars=['Codigo'] + [col for col in df_remuneraciones_encoded.columns if col.startswith('Rama_Actividad_')],
                       var_name='Fecha',
                       value_name='Remuneracion')
df_remu_long['Fecha'] = pd.to_datetime(df_remu_long['Fecha'], format='%Y-%m', errors='coerce')


In [119]:
df_ipc_long = pd.melt(df_precios_limpio,
                      id_vars=['Categoria'],
                      var_name='Fecha',
                      value_name='Variacion_IPC')
df_ipc_long['Fecha'] = pd.to_datetime(df_ipc_long['Fecha'], format='%Y-%m', errors='coerce')


In [120]:
df_tarjetas_credito['Fecha'] = pd.to_datetime(df_tarjetas_credito['Fecha'], errors='coerce')


In [121]:
# Merge entre IPC y tarjetas
df_ipc_tarjetas = pd.merge(df_ipc_long, df_tarjetas_credito, on='Fecha', how='inner')

# Merge con remuneraciones (requiere decidir cómo unir según "Rama" o "Categoria")
df_final = pd.merge(df_ipc_tarjetas, df_remu_long, on='Fecha', how='inner')


In [122]:
print(df_remu_long['Fecha'].dtype)
print(df_ipc_long['Fecha'].dtype)
print(df_tarjetas_credito['Fecha'].dtype)


datetime64[ns]
datetime64[ns]
datetime64[ns]


In [123]:
fechas_comunes = set(df_remu_long['Fecha']) & set(df_ipc_long['Fecha']) & set(df_tarjetas_credito['Fecha'])
print(sorted(fechas_comunes))


[]


In [124]:
df_remu_long['Fecha'] = pd.to_datetime(df_remu_long['Fecha']).dt.to_period('M').dt.to_timestamp()
df_ipc_long['Fecha'] = pd.to_datetime(df_ipc_long['Fecha']).dt.to_period('M').dt.to_timestamp()
df_tarjetas_credito['Fecha'] = pd.to_datetime(df_tarjetas_credito['Fecha']).dt.to_period('M').dt.to_timestamp()


In [125]:
fechas_comunes = set(df_remu_long['Fecha']) & set(df_ipc_long['Fecha']) & set(df_tarjetas_credito['Fecha'])
print(sorted(fechas_comunes))


[Timestamp('2017-12-01 00:00:00'), Timestamp('2018-01-01 00:00:00'), Timestamp('2018-02-01 00:00:00'), Timestamp('2018-03-01 00:00:00'), Timestamp('2018-04-01 00:00:00'), Timestamp('2018-05-01 00:00:00'), Timestamp('2018-06-01 00:00:00'), Timestamp('2018-07-01 00:00:00'), Timestamp('2018-08-01 00:00:00'), Timestamp('2018-09-01 00:00:00'), Timestamp('2018-10-01 00:00:00'), Timestamp('2018-11-01 00:00:00'), Timestamp('2018-12-01 00:00:00'), Timestamp('2019-01-01 00:00:00'), Timestamp('2019-02-01 00:00:00'), Timestamp('2019-03-01 00:00:00'), Timestamp('2019-04-01 00:00:00'), Timestamp('2019-05-01 00:00:00'), Timestamp('2019-06-01 00:00:00'), Timestamp('2019-07-01 00:00:00'), Timestamp('2019-08-01 00:00:00'), Timestamp('2019-09-01 00:00:00'), Timestamp('2019-10-01 00:00:00'), Timestamp('2019-11-01 00:00:00'), Timestamp('2019-12-01 00:00:00'), Timestamp('2020-01-01 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-03-01 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('20

In [126]:
# Unir IPC y tarjetas
df_ipc_tarjetas = pd.merge(df_ipc_long, df_tarjetas_credito, on='Fecha', how='inner')

# Unir lo anterior con remuneraciones
df_final = pd.merge(df_ipc_tarjetas, df_remu_long, on='Fecha', how='inner')

# Ver una muestra
print(df_final.head())
print(df_final.columns)


       Categoria      Fecha Variacion_IPC  Cantidad       Monto nominal  \
0  Nivel general 2017-12-01          23.5  78243669  98697152418.107941   
1  Nivel general 2017-12-01          23.5  78243669  98697152418.107941   
2  Nivel general 2017-12-01          23.5  78243669  98697152418.107941   
3  Nivel general 2017-12-01          23.5  78243669  98697152418.107941   
4  Nivel general 2017-12-01          23.5  78243669  98697152418.107941   

  Monto en pesos dic.2016 Codigo  Rama_Actividad_ Actividades de informática   \
0       79087045070.58577      A                                            0   
1       79087045070.58577      1                                            0   
2       79087045070.58577      2                                            0   
3       79087045070.58577      B                                            0   
4       79087045070.58577      5                                            0   

   Rama_Actividad_ Agencias de empleo eventual   \
0          

In [127]:
df_final.head(20)

,Categoria,Fecha,Variacion_IPC,Cantidad,Monto nominal,Monto en pesos dic.2016,Codigo,Rama_Actividad_ Actividades de informática,Rama_Actividad_ Agencias de empleo eventual,Rama_Actividad_ Agricultura y ganadería,...,Rama_Actividad_HOTELERIA Y RESTAURANTES,Rama_Actividad_INDUSTRIA MANUFACTURERA,Rama_Actividad_INTERMEDIACION FINANCIERA Y OTROS SERVICIOS FINANCIEROS,Rama_Actividad_PESCA Y SERVICIOS CONEXOS,"Rama_Actividad_SERVICIOS COMUNITARIOS, SOCIALES Y PERSONALES N.C.P.","Rama_Actividad_SERVICIOS DE TRANSPORTE, DE ALMACENAMIENTO Y DE COMUNICACIONES","Rama_Actividad_SERVICIOS INMOBILIARIOS, EMPRESARIALES Y DE ALQUILER",Rama_Actividad_SERVICIOS SOCIALES Y DE SALUD,Rama_Actividad_Servicios n.c.p.,Remuneracion
0,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,A,0,0,0,...,0,0,0,0,0,0,0,0,0,29477
1,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,1,0,0,1,...,0,0,0,0,0,0,0,0,0,30861
2,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,2,0,0,0,...,0,0,0,0,0,0,0,0,0,22224
3,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,B,0,0,0,...,0,0,0,1,0,0,0,0,0,72125
4,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,5,0,0,0,...,0,0,0,0,0,0,0,0,0,72125
5,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,C,0,0,0,...,0,0,0,0,0,0,0,0,0,140461
6,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,11,0,0,0,...,0,0,0,0,0,0,0,0,0,142378
7,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,13,0,0,0,...,0,0,0,0,0,0,0,0,0,s
8,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,14,0,0,0,...,0,0,0,0,0,0,0,0,0,37632
9,Nivel general,2017-12-01,23.5,78243669,98697152418.107941,79087045070.58577,D,0,0,0,...,0,1,0,0,0,0,0,0,0,112089
